In [ ]:
from tensorflow import keras
from tensorflow.python.keras import layers,Sequential,optimizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.optimizers import Adam

In [ ]:
from solve_cudnn_error import *
solve_cudnn_error()

In [ ]:
(x,y),(x_test,y_test) = keras.datasets.cifar10.load_data()

In [ ]:
plt.figure()
plt.imshow(x[0])
plt.colorbar()

In [ ]:
vgg_layers = [
    #cnn part
    #stack1
    layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
     #stack1
    layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
     #stack1
    layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
     #stack1
    layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
     #stack1
    layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
]

fc_layers =[
    layers.Dense(256,activation=tf.nn.relu),
    layers.Dense(256,activation=tf.nn.relu),
    layers.Dense(10,activation=None)
]

In [ ]:
print(type(x))
vgg = Sequential(vgg_layers)
vgg.build(input_shape=[None,32,32,3])
fc= Sequential(fc_layers)
fc.build(input_shape=[None,512])

optimizer = tf.keras.optimizers.Adam(lr=1e-4)

In [ ]:
def feature_scale(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.
    y = tf.cast(y,dtype=tf.int32)
    print(type(x))
    print(type(y))
    return x,y

In [ ]:
data = tf.data.Dataset.from_tensor_slices((x,y))
data = data.map(feature_scale).shuffle(10000).batch(512)

data_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
data_test = data_test.map(feature_scale).batch(512) 

In [ ]:
#測試選代
data_iter = iter(data)
samples = next(data_iter)
#print(samples)
print(samples[0].shape,samples[1].shape)
x,y = samples

In [ ]:
variables = vgg.variables + fc.variables
for i in range(2):
    for step,(x,y) in enumerate(data):
        with tf.GradientTape() as tape:
            logits = vgg(x)
            logits = tf.reshape(logits,[-1,512])
            logits = fc(logits)
            y_one_hot = tf.one_hot(y,depth=10)
            loss = tf.losses.categorical_crossentropy(y_one_hot,logits,from_logits=True)
            loss = tf.reduce_mean(loss)
        grads=tape.gradient(loss,variables)
        optimizer.apply_gradients(zip(grads,variables))
        #print(step)
        if step %100==0:
            print(i,step,'loss:',float(loss))
        total_loss = 0
        total_num=0
    for x,y in data_test:
        logits = vgg(x)
        logits = tf.reshape(logits,[-1,512])

        logits = fc(logits)
        prob = tf.nn.softmax(logits,axis=1)
        pred = tf.argmax(prob,axis=1)

        pred = tf.cast(pred,dtype=tf.int32)
        correct = tf.equal(pred,y)

        result = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))

        total_loss += int(result)
        total_num += x.shape[0]

    acc = total_loss/total_num
    print(i,'acc:',acc)
tf.keras.models.save_model(vgg,'vgg.h5')
tf.keras.models.save_model(fc,'fc.h5')

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

In [ ]:
!pip install tensorflow-gpu==2.0.0-beta1

In [ ]:
#測試
vgg = tf.keras.models.load_model('vgg.h5')
fc= tf.keras.models.load_model('fc.h5')
x_test=tf.cast(x_test,dtype=tf.float32)/255.
logits=vgg(x_test)
logits = tf.reshape(logits,[-1,512])
logits = fc(logits)
className=['airplane','automobile','bird','cat','deer','dog'
          ,'frog','horse','ship','truck']
i=0
for i in range(25):
    print(y_test[i])
    #prob = tf.nn.softmax(logits,axis=1)
    #print(prob)
  
    #print(logits.shape)
    #print(className[y[i]])
    #print(np.argmax())
    i+=1

In [ ]:
for i in range(2):
    for step,(x,y) in enumerate(data):
        with tf.GradientTape() as tape:
            print(step)

In [ ]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
solve_cudnn_error()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [ ]:
#tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/device:GPU:0'):
  a = tf.constant([[1.0, 2.0,3], [4.0, 5.0,7]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)